### Undefined behavior

Что такое C++ abstract machine:
* [CppCon 2020: Bob Steagall "Back to Basics: The Abstract Machine" - обстоятельный рассказ про C++ abstract machine с примерами](https://www.youtube.com/watch?v=ZAji7PkXaKY)

Определение UB в стандарте:
* https://en.cppreference.com/w/cpp/language/ub

Статьи про UB от разработчиков компилятора clang:
* [What every c programmer should know about UB. Part 1](http://blog.llvm.org/2011/05/what-every-c-programmer-should-know.html)
* [What every c programmer should know about UB. Part 2](http://blog.llvm.org/2011/05/what-every-c-programmer-should-know_14.html)
* [What every c programmer should know about UB. Part 3](http://blog.llvm.org/2011/05/what-every-c-programmer-should-know_21.html)

Толковые доклады про UB на cppcon:
* [CppCon 2019: Timur Doumler “Type punning in modern C++”](https://www.youtube.com/watch?v=_qzMpk-22cc)
* [CppCon 2019: Miro Knejp “Non-conforming C++: the Secrets the Committee Is Hiding From You”](https://www.youtube.com/watch?v=IAdLwUXRUvg)
* [CppCon 2017: Piotr Padlewski “Undefined Behaviour is awesome!”](https://www.youtube.com/watch?v=ehyHyAIa5so)


Прочее:
* [Как использование UB с signed integer ovreflow позволяет компилятору генерировать более быстрый код обработки циклов](https://gist.github.com/rygorous/e0f055bfb74e3d5f0af20690759de5a7)

<br />

##### ill-formed / well-formed program

* __ill-formed__ программа - содержит синтаксические или диагностируемые семантические ошибки. Компилятор должен выдать на такие программы сообщение (error или warning).


* __ill-formed no diagnostic required__ программа - содержит недиагностируемые семантические ошибки (либо их вычислительно дорого диагностировать). Пример: ODR violation нельзя определить во время компиляции.


__Замечание__: на уровне стандарта закреплено, что компилятор имеет false-positive ошибки при ответе на вопрос "это программа?".

Т.е. стандарт в некоторых случаях разрешает компилятору компилировать тексты, не являющиеся программами с точки зрения стандарта.

(Потому что не все ошибки диагностируемы)  

<br />

##### Разные степени ответственности компиляторов

* __implementation-defined behavior__ - разрешается различное поведение стандартом. Конкретное поведение строго документируется реализацией (компилятор + его опции, архитектура, стандартная библиотека), и в одних реализациях всегда одинакого. Примеры:
    * тип `std::size_t`
    * размер `int`
    * какую строку возвращает `std::bad_alloc::what`
    
    
* __unspecified behavior__ - разрешается различное поведение стандартом. Поведение может отличаться даже в рамках одной и той же реализации. Классический пример:
    * порядок вычисления аргументов функции (при повторном вызове порядок уже может быть иной)


* __undefined behavior__ - компилятор / программа не несёт никакой ответственности за то, что будет происходить дальше. Компиляторы не обязаны диагностировать undefined behavior. Примеры:
    * доступ за границы массива
    * арифметика указателей за границами массива
    * операции над указателем на удалённый объект
    * переполнение знаковых целых
    * разыменование нулевого указателя
    * доступ до объекта через указатель другого типа
    * использование неинициализированной переменной (`int x; std::cout << x;`)
    * побитовый сдвиг больше чем размер типа (`uint32_t x = 1; x << 32;`). Почему так:
        * X86 truncates 32-bit shift amount to 5 bits (so a shift by 32-bits is the same as a shift by 0-bits)
        * but PowerPC truncates 32-bit shift amounts to 6 bits (so a shift by 32 produces zero)
    * ...

Древняя пасхалка в gcc: если компилятор встречал определённого типа конструкцию в коде (то ли undefined, то ли implementation defined), то он запускал игрушку

https://feross.org/gcc-ownage/

<br />

* implementation defined - иди читай документацию
* unspecified - иди читай код, убеждайся, что всё нормально
* undefined - срочно править

<br />

__Вопросы для обсуждения__:

* зачем нужен undefined behavior?

<details>
<summary>Подсказка</summary>
<p>

1. иногда без него нельзя. Пример: ODR во время компиляции.
2. для скорости программы. Пример: `vector::operator[]` - без проверок на выход за границы
3. для генерации более быстрого кода компилятором. Компилятор делает вид, что программист умный, "ub"-сценариев быть не может, и на базе этого применяет некоторые оптимизации. Примеры - далее.

Ответственность за проверку корректности перекладывается с инструмента на программиста

</p>
</details>

* Основная критика UB:

    * Ответственность на человеке
    * Люди... далеки от идеала
    

* Почему критика оправдана?

<details>
<summary>Подсказка</summary>
<p>

     * потому что работает принцип Парето: 10% кода съедают 90% времени
     * "выжимать" performance нужно только в 10% кода
     * в остальных 90% можно иметь медленный код без граблей
     * проблемы undefined behavior распространяются на 100% кода
     
Замечание: Rust и его unsafe-секции - попытка реализовать этот баланс. В 10% кода пишем unsafe-секции, где страх и ужас, в 90% кода пишем безопасный код.

</p>
</details>

<br />

##### Пример как UB помогает компилятору генерировать более эффективный код

Нарушение __Type Rules__ - undefined behavior. Type Rules (за хитрыми исключениями типа `char/unsigned char/std::byte`) запрещает обращаться к объектам одного типа через другой тип.

Т.е. запрещено взять массив `float`-ов как `float*`, перекастить его через `int*` и дальше работать как с `int`.

Это даёт компилятору такой код:

```c++
float *P;
void zero_array() {
    for (int i = 0; i < 10000; ++i)
        p[i] = 0.0f;
}
```

Оптимизировать в такой (в разы быстрее):

```c++
float *p;
void zero_array() {
    memset(p, 0, 40'000);
}
```

У компилятора есть гарантии через ub, что в выражении `p[i] = 0.f;` не будет перезаписан сам `p`, т.к. типы `float` и `float*` разные.

Если какой-то программист решить нарушить Type Tules:

```c++
p = (float*)&p;
zero_array();
```

то изначальный и оптимизированный варианты неэквивалентны, но понятие UB позволяет компилятору с честными голубыми глазами применять оптимизации, программист сам виноват.

<br />

##### В случае UB разный порядок оптимизаций может дать разный результат

Рассмотрим пример (упрощённая версия одного бага из linux kernel):
    
```c++
void f(int *p) {
    int dead = *p;  // unused variable
    if (p == 0)     // check against nullptr after derefence
        return;
    *p = 4;
}
```

Рассмотрим 2 оптимизации компилятора:
* Dead Code Elimination
* Redundant Null Check Elimination

__Вариант 1__:
        
* шаг 1: применили dead code elimination

```c++
void f(int *p) {
    // int dead = *p;  // deleted by the optimizer.
    if (p == 0)
        return;
    *p = 4;
}
```

* шаг 2: применили redundant null check elimination

```c++
void f(int *p) {
    if (p == 0)   // Null check is not redundant, and is kept.
        return;
    *p = 4;
}
```

__Вариант 2:__

* шаг 1: применили redundant null check elimination

```c++
void f(int *p) {
    int dead = *p;
    if (false)  // p was dereferenced by this point, so it can't be null 
        return;
    *p = 4;
}
```

* шаг 2: применили dead code elimination

```c++
void f(int *p) {
    *p = 4;
}
```

<br />

##### Пример про удаление содержимого диска

```c++
static void (*FP)() = 0;

static void impl() {
    system("rm -rf /");
}

void set() {
  FP = impl;
}

void call() {
    FP();
}
```

Т.к. разыменование нулевого указателя - UB, по-другому `FP` проинициализировать нельзя, то clang оптимизирует такой код в:

```c++
void set() {}

void call() {
    system("rm -rf /");
}
```

<br />

##### Примеры, во что может компилироваться ub

Объяснить каждый пример, почему компилятор генерит такой выхлоп

Пример:


```c++
int foo(int x) {
    return x + 1 > x; // int overflow
}
```

`-->` (gcc 9.2)

```asm
# always returns true
foo(int):
        movl    $1, %eax
        ret
```

<br />

Пример:

```c++
int f(int x, int y)
{
    if (x > 500000 && y > 500000)
        return x * y;
    return 42;
}
```

`-->` (gcc 9.2)

```c++
f(int, int):
        cmp     edi, 500000
        jle     .L3
        cmp     esi, 500000
        jle     .L3
        mov     eax, 2147483647
        ret
.L3:
        mov     eax, 42
        ret
```

<br />

Пример:

```c++
int table[4] = {};
bool exists_in_table(int v)
{
    for (int i = 0; i <= 4; i++)  // out-of-boundary access
        if (table[i] == v)
            return true;
    return false;
}
```

`-->`

```asm
# always returns true
exists_in_table(int):
        movl    $1, %eax
        ret
```

<br />

Пример:

```c++
std::size_t f(int x)
{
    std::size_t a;  // uninitialized
    if (x)  // either x nonzero or UB
        a = 42;
    return a;  // might be an access to uninitialized
}
```

`-->`

```asm
# always returns 42
f(int):
        mov     eax, 42
        ret
```

<br />

Пример:

```c++
bool p;

// access to uninitialized
if (p)
    std::puts("p is true");

// access to uninitialized
if (!p)
    std::puts("p is false");
```

output:

```
p is true
p is false
```

<br />

Пример - опровержение теоремы Ферма:
    
```c++
#include <iostream>
 
int fermat() {
  const int MAX = 1000;
  int a = 1, b = 1, c = 1;
  // Endless loop without side effects is UB
  while (1)
  {
    if (a*a*a == b*b*b + с*c*c)
        return 1;
    a++;
    if (a > MAX) { a = 1; b++; }
    if (b > MAX) { b = 1; c++; }
    if (c > MAX) { c = 1; }
  }
  return 0;
}
 
int main() {
  if (fermat())
    std::cout << "Fermat's Last Theorem has been disproved.\n";
  else
    std::cout << "Fermat's Last Theorem has not been disproved.\n";
}
```

`-->`

```
Fermat's Last Theorem has been disproved.
```

<br />

##### Пример из Quake 3

Знаменитый пример кода из quake 3. Долгое время про него говорили как умопомрачительный трюк сумрачного гения - способ ускорить вычисление $1 / \sqrt(x)$. Функция ищет решение с достаточной для геймдева точностью.

Операция (по слухам) является критичной в коде рендеринга сцены, поэтому её ускорение давало значимый прирост в FPS.

Объяснить код:

```c++
float Q_rsqrt(float number)
{
    float x2 = number * 0.5f;
    float y = number;
    int i = * (int *) &y;
    i = 0x5f3759df - (i >> 1);  // dark magic
    y = * (float *) &i;
    return y * (1.5f - (x2 * y * y));
}
```

**Вопрос:** Где здесь ошибка?

<details>
<summary>Ответ</summary>
<p>
    
**Type Rules** - запрещено обращаться к объектам одного типа через другой тип (есть тонкие исключения про `char/unsigned char/std::byte` - про них не будем говорить). Такое обращение - UB. 

В данном примере к `float`-у обращаются как будто это `int` и наоборот.
    
</p>    
</details>

<br />

Как реализовать то же самое без UB

```c++
float Q_rsqrt(float number)
{
    int i;
    float x2 = number * 0.5f;
    float y = number;
    
    static_assert(sizeof(int) == sizeof(float));
    std::memcpy(&i, &y, sizeof(float));
    
    i = 0x5f3759df - (i >> 1);  // dark magic

    static_assert(sizeof(int) == sizeof(float));
    std::memcpy(&y, &i, sizeof(float));

    return y * (1.5f - (x2 * y * y));
}
```

Или со стандарта C++20:

```c++
float Q_rsqrt(float number)
{
    float x2 = number * 0.5f;
    float y = number;
    
    int i = std::bit_cast<int>(y);

    i = 0x5f3759df - (i >> 1);  // dark magic
    
    y = std::bit_cast<float>(i);

    return y * (1.5f - (x2 * y * y));
}
```

Замедления здесь не будет.

Компиляторы достаточно хороши в оптимизации, чтобы для исправленной версии сгенерировать код, аналогичный UB-версии (если UB-версия скомпилировалась без сюрпризов).

<br />

##### Пример, почему важно понимать, что программирование на С++ - это программирование абстрактной С++ машины для компиляторов, а не битов для процессора

```c++
struct X
{
    int a;
    int b;
};

X* create_x()
{
    X* p = (X*)malloc(sizeof(X));
    p->a = 1;
    p->b = 2;
    return p;
}
```

**Вопрос:** Как ни странно, здесь UB, но в чём ошибка?

<details>
<summary>Ответ</summary>
<p>
    
С точки зрения абстрактной машины С++ объект типа X не создан:
* под объект выделена память
* но конструктор объекта не вызван
* да, конструктор ничего не делает, но ОН НЕ ВЫЗВАН!

Объектом по адресу `p` пользоваться нельзя

</p>
</details>

**Замечание**: В стандарте С++20 подправили эффект от memmove, malloc и т.д., и больше это не UB. Объяснение:
* стандарт С работает с памятью и понятием effective type
* стандарт C++ работает с понятием объекта
* поэтому некоторые С-шные трюки с памятью ведут к UB в С++
* теперь некоторые С-шные трюки с памятью стали валидны в С++

<br />

Способ (предположительно) это поправить:

```c++
struct X
{
    int a;
    int b;
};

X* create_x()
{
    char* storage = new char[sizeof(X)];
    X* p = new(storage) X;
    p->a = 1;
    p->b = 2;
    return p;
}
```

Нооооо... он работает через раз.

**Вопрос:** Почему? Вспомним лекцию про layout класса.

<details>
<summary>Ответ</summary>
<p>

У `char` alignment = 1. У `X` alignment = 4 или 8.

</p>
</details>

И это ещё не все грабли, которые ждут вас на пути (следующая грабля - нюансы арфиметики указателей в абстрактной машине С++ и UB на UB через UB)

<br />

Способ это исправить - использовать стандартный механизм, где умные люди за вас решили все проблемы:

```c++
struct X
{
    int a;
    int b;
};

X* create_x()
{
    X* p = new X;
    p->a = 1;
    p->b = 2;
    return p;
}
```

1. Вам не нужно мучиться с вопросами работы с сырой памятью и трюками с типами в ней в "нормальном" С++
2. Эти вопросы возникают, когда вы начинаете писать custom allocator
3. **НИКОГДА НЕ ПИШИТЕ CUSTOM ALLOCATOR** (см. доклады с cppcon вверху занятия, особенно от Miro Knejp про flexible member array data member)
4. Людей, способных написать custom allocator без UB, думаю, не больше 10 в мире (судя по отзывам из комитета стандартизации, далеко не каждый участник комитета способен написать custom allocator без UB)

<br />

##### Как избежать проблем с UB?
* поменять язык
* `constexpr` (в продолжении курса)
* статические анализаторы кода
* санитары (asan, msan, tsan, ubsan ...), valgrind, drmemory ... - динамические анализаторы кода
* различные встроенные отладочные механизмы. Примеры:
    * проверки корректности [внутри имплементаций STL](https://gcc.gnu.org/onlinedocs/libstdc++/manual/debug_mode.html)
    * проверки кучи на целостность после операций с аллокаторами в windows [CrtSetDbgFlag](https://docs.microsoft.com/ru-ru/cpp/c-runtime-library/reference/crtsetdbgflag?view=vs-2019)
    * windows-specific [debugging tools](https://docs.microsoft.com/en-us/windows-hardware/drivers/debugger/gflags)
* повышать свой уровень грамотности С++

<br />

__Резюме:__
* implementation defined - иди читай документацию
* unspecified - иди читай код, убеждайся, что всё нормально
* undefined - срочно править
* регулярно гонять статические анализаторы по коду (и читать их отчёты)
* регулярно гонять тесты и основные сценарии под санитарами (и читать их отчёты)
* `constexpr` to the rescue! (как `constexpr` ловит ub - поговорим в продолжении курса)